## ChatGPT

This is just some messing around.

Docs:

* how to upload images: https://platform.openai.com/docs/guides/vision
* types of models: https://stackoverflow.com/questions/75774873/openai-api-error-this-is-a-chat-model-and-not-supported-in-the-v1-completions
* token limits: https://platform.openai.com/settings/organization/limits
* models: https://platform.openai.com/docs/models/model-endpoint-compatibility

In [4]:
#!pip install openai

In [16]:
import openai
import base64
from openai import OpenAI
from PIL import Image
import numpy as np

In [27]:
def load_image(image_path):
    img = Image.open(image_path).convert('RGB')
    new_size = np.round(np.array(img.size)*0.5).astype('int')
    img = img.resize(new_size, Image.Resampling.LANCZOS)
    #img = np.array(img)
    #with open(image_path, "rb") as image_file:
    img.save('/Users/jnaiman/Downloads/tmp/tmp_img.png')
    with open('/Users/jnaiman/Downloads/tmp/tmp_img.png','rb') as image_file:
        #return base64.b64encode(img).decode("utf-8")
        return base64.b64encode(image_file.read()).decode("utf-8")

In [28]:
# get image
image_path = '/Users/jnaiman/Downloads/data_full_v2/Picture1.png'
encoded_image = load_image(image_path)

In [50]:
# setup
with open('/Users/jnaiman/.openai/key.txt','r') as f:
    api_key = f.read()

client = OpenAI(
  api_key=api_key.strip(),  # this is also the default, it can be omitted
)

#############

# Step 3: Ask a series of questions
questions = [
    "What is happening in this image?",
    "What is the main object in this image?",
    "Can you describe the colors in this image?",
    "Does this image contain any text? If so, what does it say?"
]

responses = []

for question in questions:
    # Prepare the API request
    prompt = f"I am going to show you an image. Here is the image: [Image: {encoded_image}]. Now, {question}"
    
    # Send the request to the GPT-4o API
    response = client.chat.completions.create(
        #model="gpt-4",
        #model="gpt-4o",
        model ="gpt-4o-mini",
        #model ="gpt-3.5-turbo",
        #model="gpt-3.5-turbo-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can analyze images."},
            {"role":"user", "content": [
                {
                  "type": "text",
                  "text": prompt
                },
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                  }
                }
            ]
            }
        ]
    )
    
    # Get the response from the API
    #answer = response.choices[0].text #response.choices[0].message["content"]
    answer = response.choices[0].message.content
    responses.append((question, answer))

# Step 4: Handle the responses
for question, answer in responses:
    print(f"Q: {question}\nA: {answer}\n")

Q: What is happening in this image?
A: The image appears to be a graphical representation of data, likely from an experiment or study. It features a line graph with multiple data series plotted against a common x-axis labeled "MATERIAL CORRECTION" and a y-axis labeled "BEHAVIOUR DOUBT."

Here are a few observations:

- There are four different lines, each represented by a different color and style (solid, dashed, dotted).
- The y-axis values range from 0 to about 150.
- The x-axis ranges from 2 to 14.
- It seems that these lines could be showing different measurements or responses related to how behavior doubt changes with material correction.

Overall, this graph involves a comparative analysis of how behavior doubt varies with material correction across different conditions or treatments, represented by the colored lines.

Q: What is the main object in this image?
A: The main object in the image is a line graph that illustrates data related to "BEHAVIOUR DOUBT" against "MATERIAL CORR

In [47]:
# client.chat.completions.create?

In [48]:
# payload = {
#   "model": "gpt-4o-mini",
#   "messages": [
#     {
#       "role": "user",
#       "content": [
#         {
#           "type": "text",
#           "text": "What’s in this image?"
#         },
#         {
#           "type": "image_url",
#           "image_url": {
#             "url": f"data:image/jpeg;base64,{base64_image}"
#           }
#         }
#       ]
#     }
#   ],
#   "max_tokens": 300
# }